In [29]:
def get_chunks(path):
    headers_to_split_on = [
        ('#', 'Header 1'),
        ('##', 'Header 2'),
        ('###', 'Header 3'),
        # ('###', 'Header 4')
    ]
    all_chunks = []
    for root, _, files in os.walk(path):
        for file in tqdm(files):
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                try:
                    # Загрузка файла
                    with open(file_path) as f:
                        markdown_content = f.read()
                        
                    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
                    doc_chunks = markdown_splitter.split_text(markdown_content)
                    for chunk in doc_chunks:
                        all_chunks.append((list(chunk.metadata.values())[0], chunk.page_content))
                   
    
                except Exception as e:
                    print(f'Ошибка обработки файла {file_path}: {e}')
                    continue
    return all_chunks
                
        

In [1]:
from langchain_gigachat import chat_models, embeddings, GigaChat, GigaChatEmbeddings, output_parsers, tools, utils


In [ ]:
GigaChatEmbeddings()

In [2]:
from sentence_transformers import SentenceTransformer
import os

# Попробуйте увеличить таймаут (в секундах)
os.environ['HF_TRANSFER_TIMEOUT'] = '100'  # Увеличиваем таймаут для HF
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '600'  # Увеличиваем таймаут загрузки

embedding_model = SentenceTransformer(
    model_name_or_path='sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    # Дополнительные опции для requests
    request_timeout=120
)
test_embedding = embedding_model.embed_query("FastAPI is a modern web frameworffk")
print(f"Размерность эмбеддинга: {len(test_embedding)}")

TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'request_timeout'

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
# Инициализация модели для создания эмбеддингов
# Хорошая модель для старта: all-MiniLM-L6-v2
embedding_model = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_name = 'intfloat/multilingual-e5-large',
    model_kwargs={'device': 'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}  # Нормализация для косинусного сходства
)

# Тестируем модель
test_embedding = embedding_model.embed_query("FastAPI is a modern web frameworffk")
print(f"Размерность эмбеддинга: {len(test_embedding)}")

TypeError: SentenceTransformer.__init__() got an unexpected keyword argument 'model_name'

In [7]:
client = QdrantClient(url='localhost:6333')  # Используйте "localhost:6333" для standalone-сервера

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.vectorstores import Qdrant

# Локальный режим (для тестирования)
client = QdrantClient(":memory:")  # Используйте "localhost:6333" для standalone-сервера

# Или подключение к Qdrant Cloud
# client = QdrantClient(
#     url="your-cluster-url",
#     api_key="your-api-key",
# )

# Создание коллекции (если её нет)
try:
    client.create_collection(
        collection_name="fastapi_docs",
        vectors_config=VectorParams(
            size=384,  # Должно соответствовать размерности выбранной модели эмбеддингов!
            distance=Distance.COSINE  # Косинусное расстояние для семантического сходства
        )
    )
except Exception as e:
    print(f"Коллекция可能 уже существует: {e}")

# Инициализация векторного хранилища LangChain для Qdrant
vector_store = Qdrant(
    client=client,
    collection_name="fastapi_docs",
    embeddings=embedding_model,
)

NameError: name 'embedding_model' is not defined

In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from langchain.vectorstores import Qdrant

# Локальный режим (для тестирования)
client = QdrantClient(":memory:")  # Используйте "localhost:6333" для standalone-сервера

# Или подключение к Qdrant Cloud
# client = QdrantClient(
#     url="your-cluster-url",
#     api_key="your-api-key",
# )

# Создание коллекции (если её нет)
try:
    client.create_collection(
        collection_name="fastapi_docs",
        vectors_config=VectorParams(
            size=384,  # Должно соответствовать размерности выбранной модели эмбеддингов!
            distance=Distance.COSINE  # Косинусное расстояние для семантического сходства
        )
    )
except Exception as e:
    print(f"Коллекция可能 уже существует: {e}")

# Инициализация векторного хранилища LangChain для Qdrant
vector_store = Qdrant(
    client=client,
    collection_name="fastapi_docs",
    embeddings=embedding_model,
)

NameError: name 'embedding_model' is not defined

In [ ]:
# Пример запроса
query = "How to create a POST endpoint in FastAPI?"
found_docs = vector_store.similarity_search(
    query=query,
    k=5  # Количество возвращаемых результатов
)

# Вывод результатов
for i, doc in enumerate(found_docs):
    print(f"Результат #{i+1}:")
    print(f"Контент: {doc.page_content[:200]}...")
    print(f"Источник: {doc.metadata.get('source', 'N/A')}")
    print(f"Рейтинг: {score}")  # Если используется similarity_search_with_score
    print("-" * 50)